In [ ]:
#test data 예측
y_prob=model.predict_proba(X_test)[:,1]
y_pred=(y_prob>optimal).astype(int)

In [ ]:
test_pred=pd.DataFrame(y_pred,columns=['고위험군 예측'])
idx=X_test.index
test_pred.set_index(idx,inplace=True)

In [ ]:
X_test=pd.concat([X_test,test_pred],axis=1)

In [ ]:
#성능 확인
report = classification_report(y_test, y_pred)
print("분류 보고서:\n", report)

In [ ]:
#정확도 확인
accuracy_score(y_test,y_pred)

In [ ]:
#f1 score 확인
f1_score(y_test,y_pred)

In [ ]:
#macro f1 score 확인
f1_score(y_test,y_pred,average='macro')

In [ ]:
confusion = confusion_matrix(y_test,y_pred)
confusion

In [ ]:
y_test=tree_y_test
tree_test_df=pd.concat([X_test,y_test],axis=1)

In [ ]:
#고위험군/저위험군 모델링을 위한 데이터셋 분리
up_test_df=tree_test_df[tree_test_df['고위험군 예측']==1]
low_test_df=tree_test_df[tree_test_df['고위험군 예측']==0]

In [ ]:
#고위험군 예상 고객 Ensemble Regressor 예측
up_X_test=up_test_df.drop(['고위험군 예측','사고율'],axis=1)
up_y_test=up_test_df['사고율']

y_pred = ensemble_model1.predict(up_X_test)

up_test_pred=pd.DataFrame(y_pred,columns=['예상 사고율'])
idx=up_X_test.index
up_test_pred.set_index(idx,inplace=True)

In [ ]:
#예측값에 대한 SHAP Value 값 추출
shap_values1 = up_explainer_dt(up_X_test)
shap_values2 = up_explainer_rf(up_X_test)
shap_values3 = up_explainer_gb(up_X_test)
up_ensemble_shap_values = (shap_values1 + shap_values2 + shap_values3) / 3

In [ ]:
#SHAP Value 시각화
shap.summary_plot(up_ensemble_shap_values,up_X_test)

In [ ]:
#저위험군 예상 고객 Ensemble Regressor 예측
low_X_test=low_test_df.drop(['고위험군 예측','사고율'],axis=1)
low_y_test=low_test_df['사고율']

y_pred = ensemble_model2.predict(low_X_test)

low_test_pred=pd.DataFrame(y_pred,columns=['예상 사고율'])
idx=low_X_test.index
low_test_pred.set_index(idx,inplace=True)

In [ ]:
#예측값에 대한 SHAP Value 값 추출
shap_values4 = low_explainer_dt(low_X_test)
shap_values5 = low_explainer_rf(low_X_test)
shap_values6 = low_explainer_gb(low_X_test)
low_ensemble_shap_values = (shap_values4 + shap_values5 + shap_values6) / 3

In [ ]:
#SHAP Value 시각화
shap.summary_plot(up_ensemble_shap_values,low_X_test)

In [ ]:
#결과 데이터셋 생성
pred_res=pd.concat([up_test_pred,low_test_pred])
result=pd.concat([tree_test_df,pred_res],axis=1)

In [ ]:
#최종 성능 확인을 위한 nrmse 계산
import math
rmse = math.sqrt(mean_squared_error(result['사고율'], result['예상 사고율']))
nrmse=rmse/result['사고율'].max()
nrmse

In [ ]:
#언더라이팅에 활용할 고객 대시보드를 위해 고객 shap value 데이터셋 생성
up_shap_df = pd.DataFrame(up_ensemble_shap_values.values, columns=up_X_test.columns)
idx=up_X_test.index
up_shap_df.set_index(idx,inplace=True)

In [ ]:
low_shap_df = pd.DataFrame(low_ensemble_shap_values.values, columns=low_X_test.columns)
idx=low_X_test.index
low_shap_df.set_index(idx,inplace=True)

In [ ]:
shap_df=pd.concat([up_shap_df,low_shap_df],axis=0)

In [ ]:
shap_df.to_excel("고객 shap 데이터.xlsx", index=False,encoding='cp949')